In [1]:
from crewai import Crew, Agent, Task, LLM, Process
from crewai.tools import BaseTool
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  SerperDevTool
)
from dotenv import load_dotenv
import os
# from pydantic import BaseModel
# import subprocess
from TexSoup import TexSoup
load_dotenv()
import requests
from bs4 import BeautifulSoup
import re

/Users/bahacelik/Library/Caches/pypoetry/virtualenvs/crewaitesting-Zytas3Xz-py3.13/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [2]:
gemini_llm = LLM(
    model='gemini/gemini-2.5-flash',
    temperature=0.5,
    api_key=os.getenv('GOOGLE_API_KEY'),
    timeout=300
)

#with another api key from advokatas account
gemini_llm_2 = LLM(
    model='gemini/gemini-2.5-flash',
    temperature=0.5,
    api_key=os.getenv('GOOGLE_API_KEY_2'),
    timeout=300
)

cohere_llm = LLM(
    model='command-r',
    temperature=0.5,
    api_key=os.getenv('COHERE_API_KEY')
)

search_tool = SerperDevTool()
#scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='resume.tex')

In [3]:
#Custom scraper tool for linkedin
class LinkedinScraperTool(BaseTool):
    name: str = 'BeautifulSoup Web Scraper for Linkedin'
    description: str = 'Scrapes linkedin job descriptions without the noise'
    
    def _run(self, website_url: str) -> str:
        '''
        Scrapes the website
        '''
        try:
            response = requests.get(website_url, timeout=30)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # --- Find the specific div using its class name ---
            # This is the class for the main job description on LinkedIn
            target_class = "show-more-less-html__markup"
            job_description_div = soup.find('div', class_=target_class)

            # --- Extract text only from that div ---
            if job_description_div:
                # Use .get_text() on the found div to get its content
                text_content = job_description_div.get_text(separator=' ', strip=True)
                return text_content
            else:
                return f"Error: Could not find the job description element with class '{target_class}'."
            
        except requests.exceptions.RequestException as e:
            return f"Error fetching the URL: {e}"
        except Exception as e:
            return f"An unexpected error occurred: {e}"


Linkedin_scrape_tool = LinkedinScraperTool()

In [4]:
# response = requests.get('https://www.linkedin.com/jobs/view/4264631078')
# soup = BeautifulSoup(response.content, 'html.parser')
# target_class = 'show-more-less-html__markup'
# job_description_div = soup.find('div', class_=target_class)
# text_content = job_description_div.get_text(separator=' ', strip=True)
# text_content

In [5]:
# class LatexEditorTool(BaseTool):
#     name: str = "General LaTeX Editor"
#     description: str = """
#     A powerful tool to semantically edit any part of a LaTeX (.tex) file.
#     You can select any LaTeX command or environment and perform an action on it.

#     It takes four arguments:
#     - file_path (str): The path to the input .tex file.
#     - selector (str): The name of the LaTeX command or environment to select (e.g., 'title', 'section', 'itemize').
#     - action (str): The action to perform. Must be one of: 'replace_content', 'delete_node', 'append_after'.
#     - content (str, optional): The new LaTeX content to use for the action. Required for 'replace_content' and 'append_after'.
#     """

#     def _run(self, file_path: str, selector: str, action: str, content: str = None) -> str:
#         if not os.path.exists(file_path):
#             return f"Error: The input file '{file_path}' was not found."

#         if action in ['replace_content', 'append_after'] and content is None:
#             return f"Error: Action '{action}' requires the 'content' argument."

#         try:
#             with open(file_path, 'r', encoding='utf-8') as f:
#                 original_text = f.read()

#             output_text = ""

#             # Use robust REGEX for 'append_after' to avoid parsing errors
#             if action == 'append_after':
#                 modified_text, num_replacements = re.subn(
#                     re.escape(selector),
#                     f"{selector}\n{content}",
#                     original_text,
#                     count=1
#                 )
#                 if num_replacements == 0:
#                     return f"Error: Could not find any element matching the selector '{selector}'."
#                 output_text = modified_text

#             # Use original TexSoup logic for other powerful actions
#             elif action in ['replace_content', 'delete_node']:
#                 soup = TexSoup(original_text)
#                 target_node = soup.find(selector)

#                 if not target_node:
#                     return f"Error: Could not find any element matching the selector '{selector}'."

#                 if action == 'replace_content':
#                     new_node = TexSoup(content).expr
#                     target_node.replace_with(new_node)
#                 elif action == 'delete_node':
#                     target_node.delete()
                
#                 output_text = str(soup)
            
#             else:
#                 return f"Error: Invalid action '{action}'. Must be one of 'replace_content', 'delete_node', 'append_after'."

#             # Save the result to a new file
#             output_path = file_path.replace('.tex', '_edited.tex')
#             with open(output_path, 'w', encoding='utf-8') as f:
#                 f.write(output_text)

#             return f"Successfully performed '{action}' on '{selector}' and saved the result to '{output_path}'."

#         except Exception as e:
#             # This will now primarily catch TexSoup parsing errors for replace/delete
#             return f"An unexpected error occurred: {e}"

# # Example of how to create an instance of the tool
# latex_editor_tool = LatexEditorTool()

In [6]:
#agents
# Agent 1: Job Analyst
job_analyst = Agent(
    role="Job Description Analyst",
    goal="Analyze a job description from the given URL and extract key skills, experiences, and qualifications required.",
    backstory="An expert in parsing job postings to identify what hiring managers truly want. You excel at cutting through the noise to find the core requirements of a role.",
    llm=gemini_llm,
    tools=[Linkedin_scrape_tool],
    verbose=True,
    allow_delegation=False,
)

# Agent 2: Resume Analyst
resume_analyst = Agent(
    role="LaTeX Resume Analyst",
    goal="Thoroughly read and understand the structure and content of a given LaTeX resume.",
    backstory="A meticulous resume parser with expertise in LaTeX formatting. You can deconstruct any resume to understand a candidate's career story, skills, and past experiences.",
    llm=gemini_llm_2,
    tools=[read_resume],
    verbose=True,
    allow_delegation=False
)

# Agent 3: Career Strategist
career_strategist = Agent(
    role="Resume Enhancement Strategist",
    goal="Compare the job analysis and the resume analysis to create a strategic plan" \
    "        of attack. Identify key keywords and skills that are missing from the resume and suggest specific changes.",
    backstory="A seasoned career coach who knows how to beat Applicant Tracking Systems (ATS). Your insights bridge the gap between a candidate's experience and a job's requirements, creating a perfect match.",
    llm=gemini_llm,
    verbose=True,
    allow_delegation=False
)

# Agent 4: LaTeX Resume Editor
resume_editor = Agent(
    role="Professional LaTeX Resume Editor",
    goal="Take the strategic plan from the Career Strategist and implement the changes in the original LaTeX resume, outputting a new, perfectly tailored version.",
    backstory="A master resume writer and LaTeX expert. You can seamlessly integrate new keywords and experiences into a resume, ensuring it remains professional, well-formatted, and impactful.",
    llm=gemini_llm,
    #tools=[latex_editor_tool],
    verbose=True,
    allow_delegation=False
)

In [7]:
# Task 1: Analyze the job description
task_job_analysis = Task(
    description="Analyze the job description from the URL: {job_url}. Extract the key responsibilities, required skills (technical and soft), and company culture keywords.",
    expected_output="A detailed, structured summary of the job requirements and desired candidate profile.",
    agent=job_analyst
)

# Task 2: Analyze the current resume
task_resume_analysis = Task(
    description="Read the resume file located at '{resume_path}' and create a summary of its current content, including the skills and experiences listed.",
    expected_output="A summary of the candidate's current skills, experiences, and overall professional profile based on the resume.",
    agent=resume_analyst
)

# Task 3: Generate the strategic plan
task_strategy = Task(
    description="Compare the job analysis with the resume analysis. Create a list of actionable, specific recommendations for changes to the resume. Identify missing keywords, suggest rephrasing bullet points with action verbs from the job description, and pinpoint skills to highlight.",
    expected_output="A prioritized list of changes, including exact keywords and phrases to add to the resume to align it with the job description.",
    agent=career_strategist,
    context=[task_job_analysis, task_resume_analysis] # This task depends on the first two
)

# Task 4: Rewrite the resume
task_edit_resume = Task(
    description="""Implement the strategic recommendations to modify the resume at
            '{resume_path}'. Use the editing tool to apply the changes, ensuring the 
            final output is a valid and professional LaTeX file.
            Keep the same definitions as the original file between \documentclass[a4paper]
            to % END RESUME DEFINITIONS
            DO NOT CHANGE THE STRUCTURE OF THE LATEX FILE, Just add/edit/remove text.
            only output the latex code, no other noise or thought in the final output.
    """,
    expected_output="Updated latex file that complies with the task description",
    agent=resume_editor,
    context=[task_strategy, task_resume_analysis],
    output_file='updated_resume.tex'
)

<>:25: SyntaxWarning: invalid escape sequence '\d'
<>:25: SyntaxWarning: invalid escape sequence '\d'
/var/folders/_g/nlrpl_cd5mz07kz6t_shzl9w0000gn/T/ipykernel_46604/2927961247.py:25: SyntaxWarning: invalid escape sequence '\d'
  description="""Implement the strategic recommendations to modify the resume at


In [8]:
# Define the inputs for the crew
job_inputs = {
    'job_url': 'https://www.linkedin.com/jobs/view/4264631078',
    'resume_path': 'resume.tex'
}

# Create the crew
resume_crew = Crew(
    agents=[job_analyst, resume_analyst, career_strategist, resume_editor],
    tasks=[task_job_analysis, task_resume_analysis, task_strategy, task_edit_resume],
    process=Process.sequential,
    verbose=True
)

# Kick off the crew's work
result = resume_crew.kickoff(inputs=job_inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 353134d4-ea01-48bd-91ac-c34e414d48f5                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Job Description Analyst                                                                                 │
│                                                                                                                 │
│  Task: Analyze the job description from the URL: https://www.linkedin.com/jobs/view/4264631078. Extract the     │
│  key responsibilities, required skills (technical and soft), and company culture keywords.                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/bahacelik/Library/Caches/pypoetry/virtualenvs/crewaitesting-Zytas3Xz-py3.13/lib/python3.13/site-packages/ric
h/live.py:231: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Job Description Analyst                                                                                 │
│                                                                                                                 │
│  Thought: Action: BeautifulSoup Web Scraper for Linkedin                                                        │
│                                                                                                                 │
│  Using Tool: BeautifulSoup Web Scraper for Linkedin                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"website_url\": \"https://www.linkedin.com/jobs/view/4264631078\"}"                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 89c085ff-d881-405a-8ee9-db0418de1cef                                                                     │
│  Agent: Resume Enhancement Strategist                                                                           │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional LaTeX Resume Editor                                                                        │
│                                                                                                                 │
│  Task: Implement the strategic recommendations to modify the resume at                                          │
│              'resume.tex'. Use the editing tool to apply the changes, ensuring the                              │
│              final output is a valid and professional LaTeX file.                                               │
│              Keep the same definitions as the original file between \documentclass[a4paper]                     │
│              to % END RESUME DEFINITIONS                                                                        │
│              DO NOT CHANGE THE STRUCTURE OF THE LATEX FILE, Just add/edit/remove text.                          │
│              only output the latex code, no other noise or thought in the final output.                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Professional LaTeX Resume Editor                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```latex                                                                                                       │
│  \documentclass[a4paper]{article}                                                                               │
│      \usepackage{fullpage}                                                                                      │
│      \usepackage[colorlinks=true, linkcolor=black, urlcolor=black]{hyperref}  % Hyperlinks without colored      │
│  boxes                                                                                                          │
│      \usepackage{amsmath}                                                                                       │
│      \usepackage{amssymb}                                                                                       │
│      \usepackage{textcomp}                                                                                      │
│      \usepackage[utf8]{inputenc}                                                                                │
│      \usepackage[T1]{fontenc}                                                                                   │
│      \textheight=10in                                                                                           │
│      \pagestyle{empty}                                                                                          │
│      \raggedright                                                                                               │
│      \usepackage[left=0.8in,right=0.8in,bottom=0.8in,top=0.8in]{geometry}                                       │
│                                                                                                                 │
│  \def\bull{\vrule height 0.8ex width .7ex depth -.1ex }                                                         │
│                                                                                                                 │
│  % DEFINITIONS FOR RESUME %%%%%%%%%%%%%%%%%%%%%%%                                                               │
│                                                                                                                 │
│  \newcommand{\area} [2] {                                                                                       │
│      \vspace*{-9pt}                                                                                             │
│      \begin{verse}                                                                                              │
│          \textbf{#1}   #2                                                                                       │
│      \end{verse}                                                                                                │
│  }                                                                                                              │
│                                                                                                                 │
│  \newcommand{\lineunder} {                                                                                      │
│      \vspace*{-8pt} \\                                                                                          │
│      \hspace*{-18pt} \hrulefill \\                                                                              │
│  }                                                     

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 190a783c-ce38-4d92-9d63-36cf3dafd9f6                                                                     │
│  Agent: Professional LaTeX Resume Editor                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 353134d4-ea01-48bd-91ac-c34e414d48f5                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```latex                                                                                         │
│  \documentclass[a4paper]{article}                                                                               │
│      \usepackage{fullpage}                                                                                      │
│      \usepackage[colorlinks=true, linkcolor=black, urlcolor=black]{hyperref}  % Hyperlinks without colored      │
│  boxes                                                                                                          │
│      \usepackage{amsmath}                                                                                       │
│      \usepackage{amssymb}                                                                                       │
│      \usepackage{textcomp}                                                                                      │
│      \usepackage[utf8]{inputenc}                                                                                │
│      \usepackage[T1]{fontenc}                                                                                   │
│      \textheight=10in                                                                                           │
│      \pagestyle{empty}                                                                                          │
│      \raggedright                                                                                               │
│      \usepackage[left=0.8in,right=0.8in,bottom=0.8in,top=0.8in]{geometry}                                       │
│                                                                                                                 │
│  \def\bull{\vrule height 0.8ex width .7ex depth -.1ex }                                                         │
│                                                                                                                 │
│  % DEFINITIONS FOR RESUME %%%%%%%%%%%%%%%%%%%%%%%                                                               │
│                                                                                                                 │
│  \newcommand{\area} [2] {                                                                                       │
│      \vspace*{-9pt}                                                                                             │
│      \begin{verse}                                                                                              │
│          \textbf{#1}   #2                                                                                       │
│      \end{verse}                                                                                                │
│  }                                                                                                              │
│                                                                                                                 │
│  \newcommand{\lineunder} {                                                                                      │
│      \vspace*{-8pt} \\                                                                                          │
│      \hspace*{-18pt} \hrulefill \\                    